<a href="https://colab.research.google.com/github/ilyx02/ML/blob/main/DL_lr_3_(yolo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import yaml
import cv2
import json
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd

# Загрузка аннотаций
annotations_path = '/content/train_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)


categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}


target_category_ids = {0, 1, 2, 3, 4, 5}
unique_image_ids = set()


data = []
for ann in coco_data['annotations']:
    category_id = ann['category_id']
    if category_id in target_category_ids:
        image_id = ann['image_id']
        file_name = images.get(image_id, f"unknown_{image_id}")
        unique_image_ids.add(file_name)

        x, y, w, h = ann['bbox']
        obj_record = [categories[category_id], x, y, w, h]

        found = False
        for entry in data:
            if entry['image_id'] == file_name:
                entry['objects'].append(obj_record)
                found = True
                break

        if not found:
            data.append({'image_id': file_name, 'objects': [obj_record]})


df = pd.DataFrame(data)
df['objects'] = df['objects'].apply(lambda x: str(x).replace("'", '"'))

# Вывод количества уникальных фотографий
num_unique_images = len(unique_image_ids)
print(f"Количество уникальных фотографий с интересующими категориями: {num_unique_images}")

Количество уникальных фотографий с интересующими категориями: 754


In [ ]:
annotations_path = '/content/train_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

unique_categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
num_categories = len(unique_categories)

print(f"Количество уникальных категорий: {num_categories}")
print("Список категорий:")
for category_id, category_name in unique_categories.items():
    print(f"ID: {category_id}, Название: {category_name}")

Количество уникальных категорий: 6
Список категорий:
ID: 0, Название: damage
ID: 1, Название: corrosion
ID: 2, Название: lightning
ID: 3, Название: lightning receptor
ID: 4, Название: missing teeth
ID: 5, Название: patch


# Колво фоток по категориям на train

In [ ]:
annotations_path = '/content/train_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

category_image_counts = {cat_id: set() for cat_id in categories.keys()}

for ann in coco_data['annotations']:
    category_id = ann['category_id']
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")

    if category_id in category_image_counts:
        category_image_counts[category_id].add(file_name)

print("Количество уникальных фотографий по категориям:")
for cat_id, file_names in category_image_counts.items():
    num_unique_images = len(file_names)
    category_name = categories[cat_id]
    print(f"ID: {cat_id}, Название: {category_name}, Количество фотографий: {num_unique_images}")

Количество уникальных фотографий по категориям:
ID: 0, Название: damage, Количество фотографий: 0
ID: 1, Название: corrosion, Количество фотографий: 534
ID: 2, Название: lightning, Количество фотографий: 31
ID: 3, Название: lightning receptor, Количество фотографий: 82
ID: 4, Название: missing teeth, Количество фотографий: 372
ID: 5, Название: patch, Количество фотографий: 76


# Колво фоток по категориям на валидации

In [ ]:
annotations_path = '/content/valid_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

category_image_counts = {cat_id: set() for cat_id in categories.keys()}

for ann in coco_data['annotations']:
    category_id = ann['category_id']
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")

    if category_id in category_image_counts:
        category_image_counts[category_id].add(file_name)

print("Количество уникальных фотографий по категориям:")
for cat_id, file_names in category_image_counts.items():
    num_unique_images = len(file_names)
    category_name = categories[cat_id]
    print(f"ID: {cat_id}, Название: {category_name}, Количество фотографий: {num_unique_images}")

Количество уникальных фотографий по категориям:
ID: 0, Название: damage, Количество фотографий: 0
ID: 1, Название: corrosion, Количество фотографий: 134
ID: 2, Название: lightning, Количество фотографий: 9
ID: 3, Название: lightning receptor, Количество фотографий: 10
ID: 4, Название: missing teeth, Количество фотографий: 81
ID: 5, Название: patch, Количество фотографий: 16


# Аугментация

In [ ]:

annotations_path = '/content/drive/MyDrive/CNN /Wind Turbine damage/train/_annotations.coco.json'
images_folder = '/content/drive/MyDrive/CNN /Wind Turbine damage/train/'
output_folder = '/content/drive/MyDrive/CNN /Wind Turbine damage/augmented_train/'

os.makedirs(output_folder, exist_ok=True)

with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

data = []
for ann in coco_data['annotations']:
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")
    category = categories.get(ann['category_id'], "unknown")
    x, y, w, h = ann['bbox']
    obj_record = [category, x, y, w, h]

    found = False
    for entry in data:
        if entry['image_id'] == file_name:
            entry['objects'].append(obj_record)
            found = True
            break

    if not found:
        data.append({'image_id': file_name, 'objects': [obj_record]})

df = pd.DataFrame(data)
df['objects'] = df['objects'].apply(lambda x: str(x).replace("'", '"'))

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.2),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_names']))

def augment_image(row, augmentations_per_image=4):
    new_rows = []
    image_path = os.path.join(images_folder, row['image_id'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    objects = ast.literal_eval(row['objects'])
    bboxes = [obj[1:] for obj in objects]
    category_names = [obj[0] for obj in objects]

    for i in range(augmentations_per_image):
        try:
            transformed = transform(
                image=image,
                bboxes=bboxes,
                category_names=category_names
            )

            new_filename = f"aug{i}_{row['image_id']}"
            cv2.imwrite(os.path.join(output_folder, new_filename),
                        cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))

            new_objects = [
                [cat, *bbox]
                for bbox, cat in zip(transformed['bboxes'], category_names)
            ]

            new_rows.append({
                'image_id': new_filename,
                'objects': str(new_objects).replace("'", '"')
            })
        except Exception as e:
            print(f"Ошибка при аугментации {row['image_id']}: {str(e)}")

    return new_rows

augmented_data = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    augmented_data.append(row.to_dict())
    augmented_data.extend(augment_image(row))

augmented_df = pd.DataFrame(augmented_data)

augmented_df.to_csv(os.path.join(output_folder, 'augmented_annotations.csv'), index=False)

print(f"Оригинальный датасет: {len(df)} изображений")
print(f"Аугментированный датасет: {len(augmented_df)} изображений")
print("\nПример аугментированных данных:")
print(augmented_df.head())

In [ ]:
#  данные после аугментации
category_counts = {
    "corrosion": 0,
    "lightning": 0,
    "lightning receptor": 0,
    "missing teeth": 0,
    "patch": 0
}

for obj_list in augmented_df['objects']:
    objects = ast.literal_eval(obj_list)

    for obj in objects:
        category_name = obj[0]
        if category_name in category_counts:
            category_counts[category_name] += 1

# Выводим результаты
for category_id, category_name in zip(range(1, len(category_counts) + 1), category_counts.keys()):
    print(f"ID: {category_id}, Название: {category_name}, Количество: {category_counts[category_name]}")

ID: 1, Название: corrosion, Количество: 33621
ID: 2, Название: lightning, Количество: 154
ID: 3, Название: lightning receptor, Количество: 462
ID: 4, Название: missing teeth, Количество: 7951
ID: 5, Название: patch, Количество: 1179


In [ ]:
!cp -r "/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup" "/content/"

cp: cannot stat '/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup': No such file or directory


In [ ]:
augmented_df = pd.read_csv('/content/drive/MyDrive/Wind Turbine damage/ augmented_df_full.csv')
df_valid = pd.read_csv('/content/drive/MyDrive/Wind Turbine damage/ df_valid.csv')

In [ ]:
base_path = '/content/drive/MyDrive/Wind Turbine damage/yolo_wind_turbine_backup'

In [ ]:
# Создаем YAML файл конфигурации
yaml_content = {
    'path': base_path,
    'train': 'images/train',
    'val': 'images/val',
    'names': ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']
}

with open(os.path.join(base_path, 'wind_turbine.yaml'), 'w') as f:
    yaml.dump(yaml_content, f)

print("Подготовка данных завершена!")

Подготовка данных завершена!


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [ ]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:

if torch.cuda.is_available():
    device = 'cuda'
    device_id = '0'  # Используем первый GPU
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
else:
    device = 'cpu'
    device_id = 'cpu'
    print("Используется CPU")


base_path = '/content/drive/MyDrive/CNN /Wind Turbine damage/yolo_wind_turbine_backup'
train_images_folder = '/content/drive/MyDrive/Wind Turbine damage/yolo_wind_turbine_backup/images/train'
valid_images_folder = '/content/drive/MyDrive/Wind Turbine damage/yolo_wind_turbine_backup/images/val'

model = YOLO('yolov8n.pt')

try:
    results = model.train(
        data=os.path.join('/content/drive/MyDrive/Wind Turbine damage/yolo_wind_turbine_backup/wind_turbine.yaml'),
        epochs=30,
        batch=64,
        imgsz=640,
        device=device_id
    )
except Exception as e:
    print(f"Ошибка при обучении: {e}")

Используется GPU: Tesla T4
Ultralytics 8.3.123 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Wind Turbine damage/yolo_wind_turbine_backup/wind_turbine.yaml, epochs=30, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fa

train: Scanning /content/drive/.shortcut-targets-by-id/1RKiGuI64aK1XUX8lCikv7B-eqpoXtnTy/Wind Turbine damage/yolo_wind_turbine_backup/labels/train.cache... 2107 images, 19 backgrounds, 0 corrupt: 100%|██████████| 2107/2107 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 5.2±7.5 ms, read: 98.0±114.5 MB/s, size: 806.6 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1RKiGuI64aK1XUX8lCikv7B-eqpoXtnTy/Wind Turbine damage/yolo_wind_turbine_backup/labels/val.cache... 185 images, 86 backgrounds, 0 corrupt: 100%|██████████| 271/271 [00:00<?, ?it/s]


...

                   all        271       2619      0.459      0.341      0.325      0.151



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      27/30      9.69G      2.262      1.349     0.9641        433        640: 100%|██████████| 33/33 [19:09<00:00, 34.82s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:12<00:00,  4.09s/it]


                   all        271       2619      0.385      0.235      0.286      0.134

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      28/30      11.2G      2.289      1.354     0.9573        531        640:  48%|████▊     | 16/33 [10:19<12:39, 44.70s/it]

# Предсказание

In [ ]:
best_model = YOLO(os.path.join('runs', 'detect', 'train6', 'weights', 'best.pt'))

In [ ]:

test_folder = '/content/drive/MyDrive/Wind Turbine damage/test'

results = []

for img_name in os.listdir(test_folder):
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    predictions = best_model.predict(img, conf=0.25)

    img_objects = []
    for pred in predictions:
        for box in pred.boxes:
            class_id = int(box.cls)
            class_name = best_model.names[class_id]
            conf = float(box.conf)
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            w = x2 - x1
            h = y2 - y1

            img_objects.append([class_name, x1, y1, w, h, conf])

    results.append({
        "image_id": img_name,
        "objects": img_objects
    })

df_results = pd.DataFrame(results)